In [0]:
from pyspark.sql import functions as F

silver_path = 'bikestore.silver'
gold_path = 'bikestore.gold'

# Subconsulta "pending"
pending = (
    spark.table(silver_path+".orders")
    .filter(F.lower(F.col("status")) == "pending")
    .groupBy("customer_id", "store_name", "order_date")
    .agg(F.sum("quantity").alias("quantity"))
)

# Tabela de clientes
customers = spark.table(silver_path+".customers")

# Join e seleção final
df_orders_pending = (
    pending
    .join(customers, pending.customer_id == customers.customer_id, "left")
    .filter(customers.email.isNotNull())
    .filter(customers.phone.isNotNull())
    .select(
        pending["*"],
        customers.first_name.alias("first_name_customer"),
        customers.email,
        customers.phone
    )
)

df_orders_pending.write.mode("overwrite").\
                    option('mergeSchema', 'true').\
                    saveAsTable(gold_path+".orders_pending")